In [1]:
#!pip install openpyxl

import openpyxl
import pandas as pd
import os
import re

On va procéder ici au merging des bases de données. 

## Préparation des sites 

### Préparation de la base manuelle

In [2]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier_excel = os.path.join(base_dir, 'data', 'text', 'manuel.xlsx')

# Chargez le fichier Excel dans un DataFrame
sites = pd.read_excel(fichier_excel, sheet_name=0)

In [3]:
#On garde les colonnes intéressantes 
sites = sites[['Id', 'nom', 'site', 'Idéologie']]
sites = sites.dropna(subset=['nom'])
sites.head()

,Id,nom,site,Idéologie
0,1,Renaissance Catholique,https://renaissancecatholique.fr,Intégrisme Catholique
1,2,AGRIF,https://www.lagrif.fr/,"Islamophobie, Nationalisme"
2,3,Génération Zemmour,https://www.generation-zemmour.fr,"Nationaliste, Identitaire"
3,4,ISSEP,https://www.issep.fr/,Extrême-Droite
4,5,Egalité et Réconciliation,https://www.egaliteetreconciliation.fr,"Antisémitisme, Identitaire"


In [4]:
articles = pd.read_excel(fichier_excel, sheet_name=1) #Récupération articles
articles = articles.dropna(subset=['contenu']) #Fuck les NaN
nouveaux_noms = {
    'URL ': 'URL',
}
articles = articles.rename(columns=nouveaux_noms)

### Adaptation de Femellistes 

In [5]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = os.path.join(base_dir, 'data', 'text', 'femelliste_articles.tsv')

# Chargez le fichier Excel dans un DataFrame
femiste = pd.read_csv(fichier ,sep='\t', encoding='utf-8')

In [6]:
femiste = femiste.drop('Catégories', axis=1, errors='ignore')
femiste['ID ASSO'] = 12 #On ajoute l'ID ASSO poue le merge futur 
femiste['Type de document'] = 'opinion' #On ajoute l'ID ASSO poue le merge futur 

# Renommer les colonnes pour faciliter la concaténation

nouveaux_noms = {
    'URL Complète': 'URL',
    'Texte Complet': 'contenu',
}

femiste = femiste.rename(columns=nouveaux_noms)
femiste.head()

,Titre,Auteur,Date,Extrait,contenu,URL,ID ASSO,Type de document
0,Le transgenrisme instrumentalise la cause inte...,Alice,06/10/2023,Notre contributrice Alice (@libre.et.impolie s...,Le transgenrisme instrumentalise la cause inte...,https://www.femelliste.com/articles-femellisme...,12,opinion
1,Comment le transgenrisme et la culture queer i...,FEMELLISTE,26/06/2023,"Sur les réseaux sociaux, nous avons remarqué q...",Comment le transgenrisme et la culture queer i...,https://www.femelliste.com/articles-femellisme...,12,opinion
2,Entretien avec le collectif “Féminicides par c...,Marguerite Stern,21/04/2023,"Elles sont quatre. Depuis 2016, elles comptabi...",Entretien avec le collectif “Féminicides par c...,https://www.femelliste.com/articles-femellisme...,12,opinion
3,Tania : mère d’une adolescente anciennement tr...,Marguerite Stern,30/03/2023,Tania (prénom modifié) est éducatrice sociale ...,Tania : mère d’une adolescente anciennement tr...,https://www.femelliste.com/articles-femellisme...,12,opinion
4,"Laetitia Ky : “Je n'ai pas de genre, j'ai un s...",Marguerite Stern,14/03/2023,On l’adore. Et comme elle vient de publier son...,"Laetitia Ky : “Je n'ai pas de genre, j'ai un s...",https://www.femelliste.com/articles-femellisme...,12,opinion


### Adaptation de Tradfem

In [7]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = os.path.join(base_dir, 'data', 'text', 'tradfem_articles.tsv')

# Chargez le fichier Excel dans un DataFrame
tradfem = pd.read_csv(fichier ,sep='\t', encoding='utf-8')
tradfem.head()

,Titre,Date,Auteur,Image URL,Texte Complet,URL Complète
0,« Accorder priorité à l’identité de genre sur ...,Il y a 13 heures 03/12/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/12/ph...,« Accorder priorité à l’identité de genre sur ...,https://tradfem.wordpress.com/2023/12/03/accor...
1,Les féminicides se multiplient au Canada… et a...,Il y a 4 jours 30/11/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/11/st...,Les féminicides se multiplient au Canada… et a...,https://tradfem.wordpress.com/2023/11/30/les-f...
2,Oscar Pistorius est libéré après avoir purgé m...,Il y a 1 semaine 24/11/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/11/ph...,Oscar Pistorius est libéré après avoir purgé m...,https://tradfem.wordpress.com/2023/11/24/oscar...
3,« Pourquoi fait-il ça? » est maintenant offert...,17/11/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/11/co...,« Pourquoi fait-il ça? » est maintenant offert...,https://tradfem.wordpress.com/2023/11/17/pourq...
4,"L’horreur, l’horreur",12/11/2023,TRADFEM,https://tradfem.files.wordpress.com/2023/11/is...,"L’horreur, l’horreurTRADFEM/12/11/2023________...",https://tradfem.wordpress.com/2023/11/12/lhorr...


In [8]:
#Ajout et suppressions de colonnes intéressantes 
tradfem = tradfem.drop(['Image URL'], axis=1, errors='ignore')
tradfem['ID ASSO'] = 13 #On ajoute l'ID ASSO poue le merge futur 
tradfem['Type de document'] = 'blog' #On ajoute l'ID ASSO poue le merge futur 

#Rename utiles 
nouveaux_noms = {
    'URL Complète': 'URL',
    'Texte Complet': 'contenu',
}

tradfem = tradfem.rename(columns=nouveaux_noms)
tradfem.head()

,Titre,Date,Auteur,contenu,URL,ID ASSO,Type de document
0,« Accorder priorité à l’identité de genre sur ...,Il y a 13 heures 03/12/2023,TRADFEM,« Accorder priorité à l’identité de genre sur ...,https://tradfem.wordpress.com/2023/12/03/accor...,13,blog
1,Les féminicides se multiplient au Canada… et a...,Il y a 4 jours 30/11/2023,TRADFEM,Les féminicides se multiplient au Canada… et a...,https://tradfem.wordpress.com/2023/11/30/les-f...,13,blog
2,Oscar Pistorius est libéré après avoir purgé m...,Il y a 1 semaine 24/11/2023,TRADFEM,Oscar Pistorius est libéré après avoir purgé m...,https://tradfem.wordpress.com/2023/11/24/oscar...,13,blog
3,« Pourquoi fait-il ça? » est maintenant offert...,17/11/2023,TRADFEM,« Pourquoi fait-il ça? » est maintenant offert...,https://tradfem.wordpress.com/2023/11/17/pourq...,13,blog
4,"L’horreur, l’horreur",12/11/2023,TRADFEM,"L’horreur, l’horreurTRADFEM/12/11/2023________...",https://tradfem.wordpress.com/2023/11/12/lhorr...,13,blog


### Observatoire de la petite sirène

In [9]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier = "../data/text/petitesirene_articles.tsv" # os.path.join(base_dir, 'data', 'text', 'petitesirene_article.tsv')

# Chargez le fichier Excel dans un DataFrame
petitesirene = pd.read_csv(fichier ,sep='\t', encoding='utf-8')

#Ajout et suppressions de colonnes intéressantes 
petitesirene.drop(['Unnamed: 0'], axis=1, errors='ignore',inplace=True)
petitesirene['ID ASSO'] = 23 #On ajoute l'ID ASSO poue le merge futur 
petitesirene['Type de document'] = 'analyse' #On ajoute l'ID ASSO poue le merge futur 

#Rename utiles 
nouveaux_noms = {
        'title': 'Titre',
        'publication_date': 'Date',
        'author': 'Auteur',
        'url': 'URL',
        'text': 'contenu'
}

petitesirene = petitesirene.rename(columns=nouveaux_noms)
petitesirene.head()

,Titre,URL,Date,Auteur,contenu,ID ASSO,Type de document
0,Papier de Jacqueline Schaeffer - n°4,https://www.observatoirepetitesirene.org/post/...,30 janv.,La petite Sirène,Jacqueline Schaeffer est psychanalyste POURQ...,23,analyse
1,Papier de Christian Flavigny - n°3,https://www.observatoirepetitesirene.org/post/...,22 janv.,La petite Sirène,"Christian Flavigny est psychiatre, pédopsychia...",23,analyse
2,Papier de François Rastier - n°2,https://www.observatoirepetitesirene.org/post/...,26 déc. 2023,La petite Sirène,François Rastier est membre du conseil scienti...,23,analyse
3,Papier de Claudio Rubiliani - n°1,https://www.observatoirepetitesirene.org/post/...,11 déc. 2023,La petite Sirène,Claudio Rubiliani est au directoire de l'OPS. ...,23,analyse


## Merging

In [10]:
df = pd.concat([articles, femiste, tradfem, petitesirene], ignore_index=True, sort=False)
df.sample(10)

,ID ASSO,URL,Type de document,ID ARTICLE,contenu,Titre,Auteur,Date,Extrait
84,13.0,https://tradfem.wordpress.com/2023/09/18/lhypo...,blog,NaN,L’hypocrisie de la marque Russell Brand – sa v...,L’hypocrisie de la marque Russell Brand – sa v...,TRADFEM,18/09/2023,NaN
34,12.0,https://www.femelliste.com/articles-femellisme...,opinion,NaN,Comment le transgenrisme et la culture queer i...,Comment le transgenrisme et la culture queer i...,FEMELLISTE,26/06/2023,"Sur les réseaux sociaux, nous avons remarqué q..."
61,12.0,https://www.femelliste.com/articles-femellisme...,opinion,NaN,"""Mme Élisabeth Borne, féministes, nous inquiét...","""Mme Élisabeth Borne, féministes, nous inquiét...",Dora Moutot & Marguerite Stern,22/08/2022,“Une campagne de communication du Planning fam...
154,13.0,https://tradfem.wordpress.com/2023/05/28/au-ro...,blog,NaN,"Au Royaume-Uni, l’influence néfaste exercée pa...","Au Royaume-Uni, l’influence néfaste exercée pa...",TRADFEM,28/05/2023,NaN
52,12.0,https://www.femelliste.com/articles-femellisme...,opinion,NaN,Qu’est-ce qu’une “TERF” ?Les basesLutte fémini...,Qu’est-ce qu’une “TERF” ?,Dora Moutot & Marguerite Stern,18/10/2022,L’acronyme “TERF” signifie “Trans Exclusionary...
122,13.0,https://tradfem.wordpress.com/2023/07/06/la-vi...,blog,NaN,La victoire juridique mercredi de la LGB Allia...,La victoire juridique mercredi de la LGB Allia...,TRADFEM,06/07/2023,NaN
126,13.0,https://tradfem.wordpress.com/2023/06/26/le-ca...,blog,NaN,Le cas étrange du sexe dit « biologique »TRADF...,Le cas étrange du sexe dit « biologique »,TRADFEM,26/06/2023,NaN
111,13.0,https://tradfem.wordpress.com/2023/07/17/extra...,blog,NaN,Extrait d’un livre peu connu d’Andrea Dworkin ...,Extrait d’un livre peu connu d’Andrea Dworkin ...,TRADFEM,17/07/2023,NaN
3,3.0,https://www.generation-zemmour.fr/le-feminisme...,Article d'opinion,4.0,Le féminisme progressiste actuel défend-il tou...,NaN,NaN,NaN,NaN
168,13.0,https://tradfem.wordpress.com/2023/05/17/trois...,blog,NaN,Trois jours incognito au congrès de l’EPATH en...,Trois jours incognito au congrès de l’EPATH en...,TRADFEM,17/05/2023,NaN


In [12]:
#Refaire des nombres de référence des articles
df['ID ARTICLE'] = range(1, len(df) + 1)
df['ID ASSO'] = df['ID ASSO'].apply(int)
df

,ID ASSO,URL,Type de document,ID ARTICLE,contenu,Titre,Auteur,Date,Extrait
0,1,https://renaissancecatholique.fr/blog/les-just...,Article d'opinion,1,"Par jugement du 23 novembre 2021, la XVIIème c...",NaN,NaN,NaN,NaN
1,3,https://www.generation-zemmour.fr/le-funeste-i...,Article d'opinion,2,Le funeste inventaire de la loi bioéthique ado...,NaN,NaN,NaN,NaN
2,3,https://www.generation-zemmour.fr/1568,Article d'opinion,3,"Rousseau, Woerth et les wokes : Quand les Lumi...",NaN,NaN,NaN,NaN
3,3,https://www.generation-zemmour.fr/le-feminisme...,Article d'opinion,4,Le féminisme progressiste actuel défend-il tou...,NaN,NaN,NaN,NaN
4,3,https://www.generation-zemmour.fr/le-pronom-ie...,Article d'opinion,5,Le pronom « iel » ou le sexe des anges\nLa thé...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
178,13,https://tradfem.wordpress.com/2023/05/09/atten...,blog,179,Attention aux pseudo-victimesTRADFEM/09/05/202...,Attention aux pseudo-victimes,TRADFEM,09/05/2023,NaN
179,23,https://www.observatoirepetitesirene.org/post/...,analyse,180,Jacqueline Schaeffer est psychanalyste POURQ...,Papier de Jacqueline Schaeffer - n°4,La petite Sirène,30 janv.,NaN
180,23,https://www.observatoirepetitesirene.org/post/...,analyse,181,"Christian Flavigny est psychiatre, pédopsychia...",Papier de Christian Flavigny - n°3,La petite Sirène,22 janv.,NaN
181,23,https://www.observatoirepetitesirene.org/post/...,analyse,182,François Rastier est membre du conseil scienti...,Papier de François Rastier - n°2,La petite Sirène,26 déc. 2023,NaN


In [14]:
#On exporte 
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers le répertoire de données
chemin_data = os.path.join(base_dir, 'data')

# Enregistrez le DataFrame dans un fichier CSV dans le répertoire data
df.to_csv(os.path.join(chemin_data, 'base_def.tsv'), sep='\t', index=False)